In [1]:
import pandas  as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import selenium
import feedparser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import requests as req
from bs4 import BeautifulSoup as bs    # ambos alias son cosa mia
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor

In [2]:
def machinelearning(feature):
    global column
    global predictdataraw
    global predictdata
    column = pd.read_csv(f'{feature}_AI_onlyv.csv').drop(['Unnamed: 0'], axis=1)
    predictdataraw = pd.read_csv(f'{feature}_AI_prediction.csv')
    predictdata = pd.read_csv(f'{feature}_AI_prediction.csv').drop(['Unnamed: 0'], axis=1)

In [5]:
predictdataraw

,Unnamed: 0,insolation_flux_earth,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius]
0,0,NaN,313.321965,326.030000,1.290000,19.00
1,1,NaN,313.321965,516.219970,1.530000,29.79
2,2,NaN,313.321965,185.840000,0.830000,11.00
3,3,NaN,308.484799,1765.038901,2.773069,0.93
4,4,NaN,311.602502,798.500000,1.660000,1.13
...,...,...,...,...,...,...
1626,4762,NaN,302.706461,1919.000000,3.053000,1.10
1627,4827,NaN,NaN,11.280000,NaN,1.26
1628,5073,NaN,NaN,28.731000,NaN,0.86
1629,5074,NaN,NaN,62.950000,NaN,0.86


In [3]:
machinelearning('insolation_flux_earth')

In [4]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631 entries, 0 to 1630
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   insolation_flux_earth          0 non-null      float64
 1   equilibrium_temperature_[k]    1233 non-null   float64
 2   orbital_period_[days]          1627 non-null   float64
 3   orbit_semi-major_axis_[au])    1442 non-null   float64
 4   stellar_radius_[solar_radius]  1631 non-null   float64
dtypes: float64(5)
memory usage: 63.8 KB


In [6]:
#pon aqui 
predictdata.drop(['insolation_flux_earth'] ,axis=1, inplace=True)
predictdata.dropna(inplace=True)

In [7]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1630
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   equilibrium_temperature_[k]    1233 non-null   float64
 1   orbital_period_[days]          1233 non-null   float64
 2   orbit_semi-major_axis_[au])    1233 non-null   float64
 3   stellar_radius_[solar_radius]  1233 non-null   float64
dtypes: float64(4)
memory usage: 48.2 KB


In [8]:
column.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3634 entries, 0 to 3633
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   insolation_flux_earth          3634 non-null   float64
 1   equilibrium_temperature_[k]    3634 non-null   float64
 2   orbital_period_[days]          3634 non-null   float64
 3   orbit_semi-major_axis_[au])    3634 non-null   float64
 4   stellar_radius_[solar_radius]  3634 non-null   float64
dtypes: float64(5)
memory usage: 142.1 KB


In [10]:
#pon aqui 
X = column.drop(['insolation_flux_earth'],axis=1)
y = column['insolation_flux_earth']
X

,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius]
0,1958.0,0.736547,0.01544,0.94
1,593.0,8.462999,0.06450,0.70
2,464.0,2.290700,0.01850,0.17
3,1898.0,1.508956,0.02752,1.11
4,1442.0,2.828042,0.04016,1.12
...,...,...,...,...
3629,835.0,11.094286,0.10100,1.01
3630,823.0,6.252972,0.06200,0.73
3631,600.0,16.068638,0.11700,0.73
3632,439.0,41.194912,0.22000,0.73


In [11]:
y

0       2657.83
1         76.64
2          8.60
3       2852.63
4        661.13
         ...   
3629     122.78
3630      76.22
3631      21.64
3632       6.16
3633     446.11
Name: insolation_flux_earth, Length: 3634, dtype: float64

In [12]:
def modelexecution(X, y, depth):
    X = StandardScaler().fit_transform(X)
    global y_test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    global model1
    model1 = RandomForestRegressor(n_estimators=200, max_depth=depth, random_state=42)
    model1.fit(X_train, y_train)
    global y_pred_rf
    y_pred_rf = model1.predict(X_test)
    global MSE_RF
    MSE_RF = mean_squared_error(y_test, y_pred_rf, squared=False)
    global model2
    model2 = LinearRegression()
    model2.fit(X_train, y_train)
    global y_pred_lr
    y_pred_lr = model2.predict(X_test)
    global MSE_LR
    MSE_LR = mean_squared_error(y_test, y_pred_lr, squared=False)
    return MSE_LR, MSE_RF, y_test

In [16]:
modelexecution(X, y, 5)

(596.9368288246496,
 677.1746279114326,
 1289       7.80
 120     1634.00
 839       46.84
 1537     146.51
 2890      52.77
          ...   
 3370      18.56
 962      144.34
 945      256.83
 462      903.86
 1593      39.30
 Name: insolation_flux_earth, Length: 1091, dtype: float64)

In [19]:
y_test[10:25]

2666     76.14
1255     66.00
2211    332.74
3200      0.56
2281    230.40
32       21.00
2932    367.26
465       1.15
925     794.78
1916    353.97
1334    236.31
17       64.00
1927     35.63
2527      8.30
1411    117.96
Name: insolation_flux_earth, dtype: float64

In [21]:
#pon aqui 
mean = column['insolation_flux_earth'].mean()
mean

427.8053990093565

In [22]:
for i in y_pred_rf[10:25]:
    print('{:.2f}'.format(i))

55.35
45.60
289.43
45.60
300.28
59.73
402.72
45.60
301.71
418.21
207.06
61.14
65.40
45.60
45.80


In [23]:
final_pred = model1.predict(predictdata)

In [24]:
for i in final_pred[:]:
    print('{:.2f}'.format(i))

31379.80
31379.80
31379.80
29290.08
29306.74
31379.80
31379.80
29286.41
31379.80
31379.80
31379.80
29290.08
29305.76
29316.11
29303.35
29286.41
31379.80
31379.80
29306.74
29306.74
29306.74
29286.41
29306.74
29290.08
29290.08
29290.08
29290.08
31379.80
29290.08
29290.08
29290.08
31379.80
31379.80
29281.62
31379.80
31379.80
31379.80
31379.80
29306.74
29292.14
31379.80
29290.08
31379.80
31379.80
31379.80
31379.80
31379.80
29306.74
29290.08
31379.80
31379.80
31379.80
29290.08
29292.14
29292.14
29290.08
29316.90
29316.90
29316.90
29292.14
29290.08
29316.90
29316.90
31379.80
29316.90
29306.74
29316.11
29305.76
29478.56
29292.14
29286.41
29306.74
29306.74
29286.41
29286.41
29275.40
29306.74
29286.41
29290.08
29290.08
29306.74
29281.62
29306.74
29281.62
29290.08
29286.41
29290.08
29281.59
29478.56
29286.41
29306.74
29290.08
29290.08
29292.14
29290.08
29305.76
29316.11
29306.74
29306.74
29306.74
29306.74
29281.62
29290.08
29303.35
29316.11
29290.08
29316.90
29316.90
29290.08
29292.14
29292.14
2

In [25]:
len(final_pred)

1233

In [26]:
predictdataraw

,Unnamed: 0,insolation_flux_earth,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius]
0,0,NaN,313.321965,326.030000,1.290000,19.00
1,1,NaN,313.321965,516.219970,1.530000,29.79
2,2,NaN,313.321965,185.840000,0.830000,11.00
3,3,NaN,308.484799,1765.038901,2.773069,0.93
4,4,NaN,311.602502,798.500000,1.660000,1.13
...,...,...,...,...,...,...
1626,4762,NaN,302.706461,1919.000000,3.053000,1.10
1627,4827,NaN,NaN,11.280000,NaN,1.26
1628,5073,NaN,NaN,28.731000,NaN,0.86
1629,5074,NaN,NaN,62.950000,NaN,0.86


In [27]:
predictdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1233 entries, 0 to 1630
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   equilibrium_temperature_[k]    1233 non-null   float64
 1   orbital_period_[days]          1233 non-null   float64
 2   orbit_semi-major_axis_[au])    1233 non-null   float64
 3   stellar_radius_[solar_radius]  1233 non-null   float64
dtypes: float64(4)
memory usage: 48.2 KB


In [30]:
#pon aqui la mas corta feature
algo = predictdataraw[(predictdataraw['orbital_period_[days]'].isnull()) | (predictdataraw['orbit_semi-major_axis_[au])'].isnull()) | (predictdataraw['stellar_radius_[solar_radius]'].isnull())|(predictdataraw['equilibrium_temperature_[k]'].isnull())]
len(algo)

398

In [31]:
predictdataraw

,Unnamed: 0,insolation_flux_earth,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius]
0,0,NaN,313.321965,326.030000,1.290000,19.00
1,1,NaN,313.321965,516.219970,1.530000,29.79
2,2,NaN,313.321965,185.840000,0.830000,11.00
3,3,NaN,308.484799,1765.038901,2.773069,0.93
4,4,NaN,311.602502,798.500000,1.660000,1.13
...,...,...,...,...,...,...
1626,4762,NaN,302.706461,1919.000000,3.053000,1.10
1627,4827,NaN,NaN,11.280000,NaN,1.26
1628,5073,NaN,NaN,28.731000,NaN,0.86
1629,5074,NaN,NaN,62.950000,NaN,0.86


In [32]:
predictdataraw.drop(algo.index, inplace=True)
predictdataraw['predicted'] = final_pred
predictdataraw

,Unnamed: 0,insolation_flux_earth,equilibrium_temperature_[k],orbital_period_[days],orbit_semi-major_axis_[au]),stellar_radius_[solar_radius],predicted
0,0,NaN,313.321965,326.030000,1.290000,19.00,31379.795326
1,1,NaN,313.321965,516.219970,1.530000,29.79,31379.795326
2,2,NaN,313.321965,185.840000,0.830000,11.00,31379.795326
3,3,NaN,308.484799,1765.038901,2.773069,0.93,29290.077908
4,4,NaN,311.602502,798.500000,1.660000,1.13,29306.735908
...,...,...,...,...,...,...,...
1623,4687,NaN,307.441550,228.776000,0.704800,0.65,29316.899408
1624,4690,NaN,311.602502,30.000000,0.170000,1.12,29306.735908
1625,4750,NaN,309.291012,1107.592300,2.720500,1.79,29281.623228
1626,4762,NaN,302.706461,1919.000000,3.053000,1.10,29306.735908


In [33]:
finaldata = predictdataraw[['Unnamed: 0', 'predicted']]
finaldata

,Unnamed: 0,predicted
0,0,31379.795326
1,1,31379.795326
2,2,31379.795326
3,3,29290.077908
4,4,29306.735908
...,...,...
1623,4687,29316.899408
1624,4690,29306.735908
1625,4750,29281.623228
1626,4762,29306.735908


In [34]:
rows = list(finaldata['Unnamed: 0'])
rows2 = list(finaldata['predicted'])
nextdata = pd.read_csv('nextdata.csv')
nextdata

,Unnamed: 0,planet_name,host_name,orbital_period_[days],orbit_semi-major_axis_[au]),planet_radius,planet_mass,planet_density,eccentricity,equilibrium_temperature_[k],ratio_of_semi-major_axis_to_stellar_radius,stellar_effective_temperature_[k],stellar_radius_[solar_radius],stellar_mass_[solar_mass],stellar_metallicity_[dex],distance_[pc],number_of_planets,insolation_flux_earth,stellar_surface_gravity,stellar_age_[gyr]
0,0,11 Com b,11 Com,326.030000,1.290000,12.10,6165.60000,19.100000,0.231000,313.321965,14.599543,4742.0,19.00,2.70,-0.350,93.1846000,1,,2.31,
1,1,11 UMi b,11 UMi,516.219970,1.530000,12.30,4684.81420,13.800000,0.080000,313.321965,11.043941,4213.0,29.79,2.78,-0.020,125.3210000,1,,1.93,1.560
2,2,14 And b,14 And,185.840000,0.830000,12.90,1525.50000,3.900000,0.000000,313.321965,16.225144,4813.0,11.00,2.20,-0.240,75.4392000,1,,2.63,4.500
3,3,14 Her b,14 Her,1765.038901,2.773069,12.60,2586.74645,7.110000,0.372689,308.484799,641.181348,5338.0,0.93,0.91,0.405,17.9323000,2,,4.45,3.900
4,4,16 Cyg B b,16 Cyg B,798.500000,1.660000,13.50,565.73740,1.260000,0.680000,311.602502,315.887759,5750.0,1.13,1.08,0.060,21.1397000,1,,4.36,7.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5260,5260,Kepler-228 d,Kepler-228,11.094286,0.101000,4.04,15.40000,1.280000,0.000000,835.000000,10.900000,6043.0,1.01,0.95,-0.030,1644.6100000,3,122.78,4.47,0.300
5261,5261,Kepler-229 b,Kepler-229,6.252972,0.062000,2.20,5.48000,2.830000,0.000000,823.000000,18.940000,5120.0,0.73,0.80,0.160,826.2650000,3,76.22,4.63,1.100
5262,5262,Kepler-229 c,Kepler-229,16.068638,0.117000,4.92,21.50000,0.992000,0.000000,600.000000,36.850000,5120.0,0.73,0.80,0.160,826.2650000,3,21.64,4.63,1.100
5263,5263,Kepler-229 d,Kepler-229,41.194912,0.220000,3.85,14.20000,1.370000,0.000000,439.000000,70.750000,5120.0,0.73,0.80,0.160,826.2650000,3,6.16,4.63,1.100


In [35]:
filtered = nextdata[nextdata['Unnamed: 0'].isin(rows)]
filtered

,Unnamed: 0,planet_name,host_name,orbital_period_[days],orbit_semi-major_axis_[au]),planet_radius,planet_mass,planet_density,eccentricity,equilibrium_temperature_[k],ratio_of_semi-major_axis_to_stellar_radius,stellar_effective_temperature_[k],stellar_radius_[solar_radius],stellar_mass_[solar_mass],stellar_metallicity_[dex],distance_[pc],number_of_planets,insolation_flux_earth,stellar_surface_gravity,stellar_age_[gyr]
0,0,11 Com b,11 Com,326.030000,1.290000,12.10000,6165.60000,19.100,0.231000,313.321965,14.599543,4742.0,19.00,2.70,-0.350,93.1846000,1,,2.31,
1,1,11 UMi b,11 UMi,516.219970,1.530000,12.30000,4684.81420,13.800,0.080000,313.321965,11.043941,4213.0,29.79,2.78,-0.020,125.3210000,1,,1.93,1.560
2,2,14 And b,14 And,185.840000,0.830000,12.90000,1525.50000,3.900,0.000000,313.321965,16.225144,4813.0,11.00,2.20,-0.240,75.4392000,1,,2.63,4.500
3,3,14 Her b,14 Her,1765.038901,2.773069,12.60000,2586.74645,7.110,0.372689,308.484799,641.181348,5338.0,0.93,0.91,0.405,17.9323000,2,,4.45,3.900
4,4,16 Cyg B b,16 Cyg B,798.500000,1.660000,13.50000,565.73740,1.260,0.680000,311.602502,315.887759,5750.0,1.13,1.08,0.060,21.1397000,1,,4.36,7.400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4687,4687,Kepler-16 b,Kepler-16,228.776000,0.704800,8.44900,105.83300,0.964,0.006900,307.441550,233.160883,4450.0,0.65,0.69,-0.300,75.0852000,1,,4.65,3.800
4690,4690,Kepler-160 d,Kepler-160,30.000000,0.170000,55.25309,100.00000,NaN,,311.602502,32.638790,5471.0,1.12,0.97,0.140,937.0130000,3,,4.47,1.600
4750,4750,Kepler-1647 b,Kepler-1647,1107.592300,2.720500,11.87400,483.00000,1.590,0.058100,309.291012,326.812645,6210.0,1.79,1.22,-0.140,1212.4500000,1,,4.02,4.400
4762,4762,Kepler-1656 c,Kepler-1656,1919.000000,3.053000,14.00000,126.40000,0.253,0.527000,302.706461,596.811622,5569.0,1.10,1.03,0.190,185.8750000,2,,4.40,6.310


In [36]:
#pon aqui 
nextdata.loc[filtered.index, 'insolation_flux_earth'] = rows2
nextdata['insolation_flux_earth'] = nextdata['insolation_flux_earth'].replace(' ', np.nan)
nextdata['insolation_flux_earth'] = nextdata['insolation_flux_earth'].astype(float)

In [37]:
nextdata.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5265 entries, 0 to 5264
Data columns (total 20 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   Unnamed: 0                                  5265 non-null   int64  
 1   planet_name                                 5265 non-null   object 
 2   host_name                                   5265 non-null   object 
 3   orbital_period_[days]                       5261 non-null   float64
 4   orbit_semi-major_axis_[au])                 5076 non-null   float64
 5   planet_radius                               5260 non-null   float64
 6   planet_mass                                 5260 non-null   float64
 7   planet_density                              5234 non-null   float64
 8   eccentricity                                5265 non-null   object 
 9   equilibrium_temperature_[k]                 4867 non-null   float64
 10  ratio_of_sem

In [38]:
nextdata.drop(['Unnamed: 0'], axis=1, inplace=True)
nextdata.to_csv('nextdata.csv')

In [39]:
nextdata.to_csv('insolationflux_backup.csv')